# <center>明星微博热点分析</center>
通过对微博热点文件（kriswu.csv)中的粉丝数量，流量数据等的统计，综合运用文件读取、pandas数据处理和matplotlib绘图功能等，全面掌握Python程序设计与数据处理方法，从而使读者具备大数据处理的基本能力。

## 1 运行环境
运行环境：python3.6
需要安装的包：pyecharts，pandas，numpy，matplotlib，seaborn，snownlp
我们选题使用的主流Notebook环境是jupyter Notebook. 在Jupyter Notebook中要使用了多个第三方模块工具
1. jieba在分析中是使用最广泛的分词模块。
2. wordcloud可以更加直观和艺术的展示文本
3. Pandas、numpy、ipython要处理大型数据集常用的Python模块。
4. Matplotlib一个Python的图形框架，用于绘制二维图形。
5. pyechars用来解决数据分析可视化的问题。
6. Seaborn是基于matplotlib的图形可视化python包。它提供了一种高度交互式界面，便于用户能够做出各种有吸引力的统计图表。

然后打开cmd命令框对以上几个第三方模块的环境进行配置安装

## 2 分析目标
1. 吴亦凡该微博的转发是否存在假流量？
2. 大家对于《大碗宽面》怎么看？
3. 有多少人拿吴亦凡跟蔡徐坤做对比？
4. 有多少人开始路转粉了？
5. 评论的词云图


## 3 分析内容
首先将明星吴亦凡的微博热点数据进行csv和txt文件读取，其次提取全部的信息去重，将信息进行数据清洗，然后提前需要的信息并进行去空处理，接着对提取的干净信息进行统计分析，最后生成相应的图且进行总结、PPT制作。

## 4 分析步骤

### 4.1 csv文件信息读取

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyecharts.charts.bar import Bar
%matplotlib inline

plt.style.use('ggplot')
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']  #解决seaborn中文字体显示问题
plt.rc('figure', figsize=(10, 10))  #把plt默认的图片size调大一点
plt.rcParams["figure.dpi"] =mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题

ModuleNotFoundError: No module named 'pyecharts'

In [ ]:
data = pd.read_csv('kriswu.csv', encoding='UTF-8')  

In [ ]:
#打印，以确保数据运行正常
data

In [ ]:
data.info()

In [ ]:
data.sample(5)

### 4.2数据清洗
由于数据入库的时候没有进行清洗，所以数据多出了很多没用的字段，需要先清洗掉

In [ ]:
print(list(data.columns))

In [ ]:
in_columns = ['attitudes_count', 'comments_count', 'reposts_count', 'mid', 'raw_text', 
          'source', 'user.description', 'user.follow_count', 'user.followers_count', 
          'user.gender', 'user.id', 'user.mbrank', 'user.mbtype', 'user.profile_url', 
          'user.profile_image_url', 'user.screen_name', 'user.statuses_count', 
          'user.urank', 'user.verified', 'user.verified_reason']
in_columns

In [ ]:
data = data[in_columns]
data

In [ ]:
data.info()

In [ ]:
data.to_csv('kriswu.csv', index=False)
data.to_csv

### 4.3 数据分析

#### 4.3.1 吴亦凡该微博的转发是否存在假流量？

In [ ]:
# 先来看看吴亦凡的粉丝性别比例
fans_num = data['user.gender'].value_counts()
fans_num

In [ ]:
from pyecharts import Bar

bar = Bar("吴亦凡粉丝性别比例初探", width = 600,height=500)
bar.add("(总数据102118条)", ['女', '男'], fans_num.values, is_stack=True, 
       xaxis_label_textsize=20, yaxis_label_textsize=14, is_label_show=True)
bar

In [ ]:
np.round(fans_num/fans_num.sum()*100, 2)

In [ ]:
data[data['user.gender']=='f'].sample(5)

In [ ]:
data[data['user.gender']=='m'].sample(5)

In [ ]:
#粉丝数大于等于5但昵称包含“用户”的这样的数据抽取出来。基本上可以断定是假粉丝
data_fake2_index = data[(data['user.follow_count']>5)&
                        (data['user.followers_count']>5)&
                        (data['user.screen_name'].str.contains('用户'))].index
data_fake2_index

In [ ]:
data_fake2_index.shape

In [ ]:
# 昵称里包含“用户”的，基本上可以断定是假粉丝
data_fake2_index = data[(data['user.follow_count']>5)&
                        (data['user.followers_count']>5)&
                        (data['user.screen_name'].str.contains('用户'))].index
data_fake2_index

In [ ]:
# 把假的流量粉丝转发组合起来
data_fake = pd.concat([ data.iloc[data_fake2_index]])
data_fake

In [ ]:
data_fake.shape

In [ ]:
# 取出真粉的转发
data_true = data.drop(data_fake.index)
data_true

In [ ]:
data_true.shape

In [ ]:
print('真粉丝转发数占总转发数的{}%'.format(np.round(data_true.shape[0]/data.shape[0]*100, 2)))
print('假粉丝转发数占总转发数的{}%'.format(np.round(data_fake.shape[0]/data.shape[0]*100, 2)))

In [ ]:
bar = Bar("吴亦凡真假流量的转发量", width = 600,height=500)
bar.add("(总数据102118条)", ['总转发量', '假粉丝转发量', '真粉丝转发量'], 
        [data.shape[0], data_fake.shape[0], data_true.shape[0]], is_stack=True, 
       xaxis_label_textsize=20, yaxis_label_textsize=14, is_label_show=True)
bar

In [ ]:
real_fans_num = data_true.drop_duplicates(subset='user.id').shape[0]
real_fans_num 

In [ ]:
bar = Bar("吴亦凡真假流量的转发量与真实转发粉丝量(总数据102118条)", width = 600,height=500)
bar.add('', ['总转发量', '假粉丝转发量', '真粉丝转发量', '真实转发粉丝量'], 
        [data.shape[0], data_fake.shape[0], data_true.shape[0], real_fans_num], is_stack=True, 
       xaxis_label_textsize=20, yaxis_label_textsize=14, is_label_show=True, xaxis_rotate=20)
bar

In [ ]:
print('真实转发粉丝量占总转发数的{}%'.format(np.round(real_fans_num/data.shape[0]*100, 2)))

#### 4.3.2大家对于《大碗宽面》怎么看？

In [ ]:
data.info()

In [ ]:
from snownlp import SnowNLP


def get_sent_snownlp(data):
    s = SnowNLP(data)
    return s.sentiments

In [ ]:
data_true['clean_text'] = data_true['raw_text'].str.split("//", expand=True)[0]
data_true['clean_text'] 

In [ ]:
data_true['sent_nlp'] = data_true.loc[data_true['clean_text']!='', 'clean_text'].apply(get_sent_snownlp)
data_true['sent_nlp']

In [ ]:
data_true[['clean_text', 'sent_nlp']].sample(5)

In [ ]:
data_true.loc[-data_true['clean_text'].isin(['转发微博', 'repost', '轉發微博']), 'sent_nlp'].mean()

In [ ]:
from pyecharts.charts.gauge import Gauge
from pyecharts.charts.geo import Geo


g = Gauge()
g.add('', ['评论对《大碗宽面》\n的平均评分'], [68.6])
g

In [ ]:
data_true.loc[data_true['sent_nlp']>0.9, ['clean_text', 'sent_nlp']].sample(5)

In [ ]:
data_true.loc[data_true['attitudes_count'].sort_values(ascending=False)[:10].index, ['user.screen_name', 'clean_text', 'attitudes_count']]

In [ ]:
data_true.loc[data_true['attitudes_count'].sort_values(ascending=False)[:100].index, ['user.screen_name', 'clean_text', 'attitudes_count']].sample(5)

#### 4.3.3 有多少人拿吴亦凡跟蔡徐坤做对比？

In [ ]:
data_true.info()

In [ ]:
data_true['clean_text'].sample(10)

In [ ]:
data_true['raw_text'].str.contains('kun|坤|律师|球|函|cxk|比|弟|CXK|胸|格局|气度|衬托').sum()

In [ ]:
data_kun = data_true.loc[data_true['raw_text'].str.contains('kun|坤|律师|球|函|cxk|比|弟|CXK|胸|格局|气度|衬托'),
             ['user.screen_name', 'raw_text', 'attitudes_count']]

In [ ]:
data_kun.loc[list(data_kun['attitudes_count'].nlargest(10).index), ['raw_text', 'attitudes_count']]

In [ ]:
data_true[data_true['raw_text']!=''].shape

In [ ]:
6338/101685

#### 4.3.4 有多少人开始路转粉了？

In [ ]:
data_true['clean_text'].sample(10)

In [ ]:
data_true['raw_text'].str.contains('转粉|爱上|重新|路|圈粉|espect|瑞思拜').sum()

In [ ]:
data_true.loc[data_true['raw_text'].str.contains('转粉|爱上|重新|路|圈粉|espect|瑞思拜'),
             ['user.screen_name', 'raw_text']].sample(5)

In [ ]:
fans = data_true.loc[data_true['raw_text'].str.contains('转粉|爱上|重新|路|圈粉|espect|瑞思拜'), 
                     'user.gender'].value_counts()

In [ ]:
from pyecharts.charts.bar import Bar
bar = Bar("路转粉的男女性别比例", width = 600,height=500)
bar.add("", ['女', '男'], fans.values, is_stack=True, 
       xaxis_label_textsize=20, yaxis_label_textsize=14, is_label_show=True)
bar

In [ ]:
fans/fans.sum()

## 5. 评论的词云图

In [ ]:
# 绘制评论词云图
import jieba
from collections import Counter
from pyecharts.charts.wordcloud import WordCloud

jieba.add_word('吴亦凡')


swords = [x.strip() for x in open ('stopwords.txt', encoding='UTF-8')]

In [ ]:
def plot_word_cloud(data, swords):
    text = ''.join(data)
    words = list(jieba.cut(text))
    ex_sw_words = []
    for word in words:
        if len(word)>1 and (word not in swords):
            ex_sw_words.append(word)
    c = Counter()
    c = Counter(ex_sw_words)
    wc_data = pd.DataFrame({'word':list(c.keys()), 'counts':list(c.values())}).sort_values(by='counts', ascending=False).head(100)
    wordcloud = WordCloud(width=1300, height=620)
    wordcloud.add("", wc_data['word'], wc_data['counts'], word_size_range=[12, 60])
    return wordcloud

In [ ]:
plot_word_cloud(data=data_true.loc[-data_true['clean_text'].str.contains("转发"), 'clean_text'], swords=swords)

## 6 总结

通过对明星吴亦凡的微博数据进行分析我们得出的总结是:女粉丝所占比例比男粉丝多，所有转发中，绝大部分是真实粉丝转发，只有少量的假粉丝转发量；大家对《大碗宽面》的评论平均分是68.6；拿吴亦凡和蔡徐坤做对比的人数通过提取数据前十条显示有6388人；最后路转粉的人也挺多的，其中男女比例大概是男：女=3：7。